In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys

In [2]:
dataset = pd.read_csv("data/train.csv")
dataset.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
dataset.shape

(404290, 6)

In [4]:
new_dataset = dataset[dataset["question1"].isnull() == False]
new_dataset = new_dataset[new_dataset["question2"].isnull() == False]

In [5]:
new_dataset.to_csv("data/cleaned_dataset.csv")

In [6]:
(new_dataset['is_duplicate'] == 1).sum(), new_dataset.shape[0], (new_dataset['is_duplicate'] == 1).sum()/new_dataset.shape[0]

(149263, 404287, 0.3692005926482919)

In [7]:
len(new_dataset['question1']), len(new_dataset['question1'].unique())

(404287, 290455)

In [8]:
len(new_dataset['question2']), len(new_dataset['question2'].unique())

(404287, 299173)

In [9]:
len(set(new_dataset["question2"].unique()).intersection(set(new_dataset["question1"].unique())))

52269

In [10]:
unique_questions = {}
ctr_idx= 0
positive_pairs = []

for row in new_dataset.iterrows():
    if row[1]['is_duplicate'] == 1:
        if not row[1]['question1'] in unique_questions:
            unique_questions[row[1]['question1']] = ctr_idx
            q1_id = ctr_idx
            ctr_idx += 1
        if not row[1]['question2'] in unique_questions:
            unique_questions[row[1]['question2']] = ctr_idx
            q2_id = ctr_idx
            ctr_idx += 1
        if q1_id != q2_id:
            positive_pairs.append([unique_questions[row[1]['question1']], unique_questions[row[1]['question1']]])
print(len(unique_questions))

149596


In [11]:
pos_pair_dict = {}
for i in positive_pairs:
    pos_pair_dict[i[0]] = i[1]
    pos_pair_dict[i[1]] = i[0]

In [12]:
unique_questions_rmap = {}
for k,v in unique_questions.items():
    unique_questions_rmap[v] = k

In [13]:
print(len(unique_questions), len(positive_pairs))

149596 149263


In [14]:
import random

In [15]:
triplets = []

for pair in positive_pairs:
    while True:
        negative = random.randint(0, len(unique_questions))
        if negative == pair[0] or negative == pair[1] or pos_pair_dict[pair[1]] == negative or pos_pair_dict[pair[0]] == negative:
            continue
        else:
            break
    triplets.append([pair[0], pair[1], negative])
    
print(len(triplets))

149263


In [16]:
random.shuffle(triplets)
len_80_20 = int(0.8 * len(triplets))
train_triples = triplets[0:len_80_20]
test_triples = triplets[len_80_20:]

In [17]:
new_dataset[['question1', 'question2', 'is_duplicate']]

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...
404285,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,What is one coin?,What's this coin?,0
404288,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


------

In [18]:
questions = np.concatenate((new_dataset["question1"].to_numpy(), new_dataset["question2"].to_numpy()))

In [19]:
questions

array(['What is the step by step guide to invest in share market in india?',
       'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
       'How can I increase the speed of my internet connection while using a VPN?',
       ..., "What's this coin?",
       'I am having little hairfall problem but I want to use hair styling product. Which one should I prefer out of gel, wax and clay?',
       'What is it like to have sex with your cousin?'], dtype=object)

In [20]:
questions.shape

(808574,)

In [21]:
import transformers

/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [22]:
transformers.__version__

'2.10.0'

In [23]:
import torch
from transformers import AutoTokenizer, AutoModel

In [24]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

In [27]:
model.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [34]:
# def embed_text_using_bert(text):
#     input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)  # Batch size 1
#     outputs = model(input_ids.to('cuda'))
#     last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
#     return last_hidden_states.cpu().mean(1).detach().numpy()

In [28]:
def embed_text_batch_using_bert(text_list):
    input_ids = torch.tensor(tokenizer.batch_encode_plus(data_batch, pad_to_max_length=True)['input_ids'])
    outputs = model(input_ids.to('cuda'))
    last_hidden_states = outputs[0].cpu()  # The last hidden-state is the first element of the output tuple
    
    # return the vec of the CLS token
    return last_hidden_states[:,0,:].detach().numpy()
    
    # return the mean of all token vecs
    #return last_hidden_states.mean(1).detach().numpy()

In [29]:
data_batch = ["How can I increase the speed of my internet connection while using a VPN?", 
              "How can I increase the speed of my internet connection"]

vecs = embed_text_batch_using_bert(data_batch)
vecs.shape

(2, 768)

### Create train-test data in the format of vocab id sequences

In [30]:
df = pd.read_csv('data/cleaned_dataset.csv')
df.head()

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate
0,0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [31]:
from pytorch_pretrained_bert import BertTokenizer

In [32]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_text = tokenizer.tokenize('How can I increase the speed of my internet connection')
tokenizer.convert_tokens_to_ids(tokenized_text)

[2129, 2064, 1045, 3623, 1996, 3177, 1997, 2026, 4274, 4434]

In [33]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [34]:
input_ids = torch.tensor(tokenizer.batch_encode_plus(data_batch, pad_to_max_length=True)['input_ids'])
input_ids

tensor([[  101,  2129,  2064,  1045,  3623,  1996,  3177,  1997,  2026,  4274,
          4434,  2096,  2478,  1037, 21210,  2078,  1029,   102],
        [  101,  2129,  2064,  1045,  3623,  1996,  3177,  1997,  2026,  4274,
          4434,   102,     0,     0,     0,     0,     0,     0]])

### Quora custom dataset class

Custom Dataset class needs to implement three functions: __init__, __len__, and __getitem__.

In [35]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [36]:
class QuoraQuestionsDataset(Dataset):
    def __init__(self, data):
        self.question_tok_triples = data
        
    def __len__(self):
        return len(self.question_tok_triples)

    def __getitem__(self, idx):
        return self.question_tok_triples[idx]
#         img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
#         image = read_image(img_path)
#         label = self.img_labels.iloc[idx, 1]
#         if self.transform:
#             image = self.transform(image)
#         if self.target_transform:
#             label = self.target_transform(label)
#         return image, label

In [ ]:
# train_dataloader = DataLoader(train_triples[0:6400], batch_size=64, shuffle=True)
# test_dataloader = DataLoader(train_triples[0:1600], batch_size=64, shuffle=True)

### Train with triplet margin loss

In [37]:
from pytorch_pretrained_bert import BertTokenizer, BertModel

In [38]:
from torch.nn import TripletMarginLoss
from torch import nn
import torch.optim as optim
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [39]:
class CLNetwork(nn.Module):
    def __init__(self, emb_dim=768):
        super(CLNetwork, self).__init__()
        #self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert = BertModel.from_pretrained('bert-base-uncased')
#         self.conv = nn.Sequential(
#             nn.Conv2d(1, 32, 5),
#             nn.PReLU(),
#             nn.MaxPool2d(2, stride=2),
#             nn.Dropout(0.3),
#             nn.Conv2d(32, 64, 5),
#             nn.PReLU(),
#             nn.MaxPool2d(2, stride=2),
#             nn.Dropout(0.3)
#         )
        
#         self.fc = nn.Sequential(
#             nn.Linear(64*4*4, 512),
#             nn.PReLU(),
#             nn.Linear(512, emb_dim)
#         )
        
    def forward(self, x):
        x = self.bert(x)
        return x

In [40]:
def init_weights(m):
    torch.nn.init.kaiming_normal_(m.weight)

In [72]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
        
    def calc_euclidean(self, x1, x2):
        return (x1 - x2).pow(2).sum()
    
    def forward(self, anchor: torch.Tensor, positive: torch.Tensor, negative: torch.Tensor) -> torch.Tensor:
        distance_positive = self.calc_euclidean(anchor, positive)
        distance_negative = self.calc_euclidean(anchor, negative)
        losses = torch.relu(distance_positive - distance_negative + self.margin)

        return losses.mean()

In [71]:
(anchor_out-anchor_out).pow(2).sum()

tensor(0., grad_fn=<SumBackward0>)

In [73]:
model = CLNetwork(768)
# model = torch.jit.script(model)#.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = TripletLoss()
# criterion = torch.jit.script(TripletLoss())

In [44]:
len(train_triples), len(test_triples)

(119410, 29853)

In [53]:
[len(train_trip_toks[i][1]) for i in range(0, 10)]

[20, 20, 20, 20, 20, 20, 20, 20, 20, 20]

In [173]:
tokenizer.batch_encode_plus??

In [47]:
batch_size=1 #16

In [48]:
train_trip_toks = []
for trip in train_triples[0:144]:
    data_batch = [unique_questions_rmap[trip[0]], unique_questions_rmap[trip[1]], unique_questions_rmap[trip[2]]]
    train_trip_toks.append(torch.tensor(tokenizer.batch_encode_plus(data_batch, max_length=20, pad_to_max_length=True)['input_ids']))
    
train_ds = QuoraQuestionsDataset(train_trip_toks)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4)

In [49]:
test_trip_toks = []
for trip in test_triples[0:80]:
    data_batch = [unique_questions_rmap[trip[0]], unique_questions_rmap[trip[1]], unique_questions_rmap[trip[2]]]
    test_trip_toks.append(torch.tensor(tokenizer.batch_encode_plus(data_batch, max_length=20, pad_to_max_length=True)['input_ids']))

test_ds = QuoraQuestionsDataset(test_trip_toks)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4)

In [50]:
from tqdm import tqdm as tqdm

In [51]:
for _, i in enumerate(tqdm(train_loader, desc="Training", leave=False)):
    print(_)
    print(i.shape)
    print(i[0][0], i[0][1], i[0][2])
    break

0
torch.Size([1, 3, 20])
tensor([ 101, 2054, 2079, 2116, 2111, 6232, 4697, 4419, 2739, 1029,  102,    0,
           0,    0,    0,    0,    0,    0,    0,    0]) tensor([ 101, 2054, 2079, 2116, 2111, 6232, 4697, 4419, 2739, 1029,  102,    0,
           0,    0,    0,    0,    0,    0,    0,    0]) tensor([ 101, 2054, 2003, 1996, 2779, 6451, 3021, 1999, 8955, 1029,  102,    0,
           0,    0,    0,    0,    0,    0,    0,    0])


In [54]:
bert = BertModel.from_pretrained('bert-base-uncased')

In [55]:
bert(test_trip_toks[0])[1]

tensor([[-0.6644, -0.3830, -0.9211,  ..., -0.7755, -0.5754,  0.7168],
        [-0.7301, -0.3435, -0.8384,  ..., -0.6896, -0.6478,  0.7646],
        [-0.7458, -0.3820, -0.9208,  ..., -0.7530, -0.5866,  0.7146]],
       grad_fn=<TanhBackward0>)

In [56]:
type(torch.tensor(tokenizer.batch_encode_plus(data_batch, max_length=20, pad_to_max_length=True)['input_ids']))
torch.tensor(tokenizer.batch_encode_plus(data_batch, max_length=20, pad_to_max_length=True)['input_ids']).shape
pp = torch.tensor(tokenizer.batch_encode_plus(data_batch, max_length=20, pad_to_max_length=True)['input_ids'])
op = bert(pp)
len(op), len(op[0]), len(op[1])

(2, 12, 3)

In [57]:
for i in range(0, 12):
    print(op[0][i].shape)


print("Last layer:")
print(op[0][-1][:,0,:])

torch.Size([3, 20, 768])
torch.Size([3, 20, 768])
torch.Size([3, 20, 768])
torch.Size([3, 20, 768])
torch.Size([3, 20, 768])
torch.Size([3, 20, 768])
torch.Size([3, 20, 768])
torch.Size([3, 20, 768])
torch.Size([3, 20, 768])
torch.Size([3, 20, 768])
torch.Size([3, 20, 768])
torch.Size([3, 20, 768])
Last layer:
tensor([[-7.6433e-02, -8.9489e-02, -1.3672e-01,  ..., -6.6198e-02,
          5.6868e-01, -1.7539e-02],
        [-3.7741e-01,  2.4381e-04,  2.9177e-02,  ..., -2.3378e-01,
          5.8122e-01, -1.1460e-01],
        [-2.2106e-01, -1.2737e-02,  2.2621e-01,  ..., -4.2140e-01,
          5.4900e-01,  2.6106e-01]], grad_fn=<SliceBackward0>)


In [273]:
print(op[0][-1][:,0,:][0])

tensor([ 1.0479e-01, -1.3670e-01,  2.8593e-01,  3.9472e-01, -2.8711e-01,
        -9.5799e-03,  4.4399e-01,  1.9117e-01, -5.9215e-02, -1.5948e-01,
         5.4828e-01, -7.1385e-02,  2.3403e-01,  8.3681e-02,  6.1395e-02,
         2.3235e-01, -9.5476e-02,  4.1060e-01,  1.9296e-01, -8.3265e-03,
         2.5403e-01,  3.4456e-01,  7.4730e-01, -8.3072e-02,  2.1567e-01,
        -2.0264e-01,  2.2041e-01, -5.8181e-01, -3.8102e-01, -3.3458e-01,
        -2.3308e-01,  4.7912e-01, -1.4395e-02, -3.1721e-01, -2.6404e-02,
        -3.0581e-01, -1.6827e-01, -3.0518e-02,  5.3785e-01, -3.6537e-01,
        -3.6258e-01, -3.0649e-02,  4.5609e-01,  9.6171e-02, -1.9731e-01,
        -1.7379e-01, -3.9627e+00,  9.1790e-02, -2.7941e-01, -2.9545e-01,
        -1.2409e-01,  6.4611e-02,  2.0612e-01, -6.6146e-03, -5.4270e-02,
         6.2808e-01, -2.9875e-01, -1.2810e-01,  7.8158e-01,  1.8447e-01,
         2.5692e-01,  2.5277e-01, -3.7966e-01,  2.1000e-01, -1.1052e-01,
         4.0831e-01, -1.5077e-01,  1.3938e-01, -3.1

In [274]:
print(op[0][-1][:,0,:][1])

tensor([-1.4680e-01, -4.5984e-01,  6.1936e-02,  3.3654e-01, -4.3310e-01,
        -3.1909e-02,  6.8949e-01,  2.8380e-01, -2.2587e-01, -1.1740e-01,
         4.5956e-01, -5.1627e-01,  3.7880e-02,  5.9976e-01,  7.3889e-02,
         1.5098e-01, -3.9343e-01,  6.3144e-01, -8.1088e-03,  4.7656e-02,
         3.2977e-01,  1.2260e-01,  6.4057e-01, -1.7358e-01, -1.1333e-01,
        -1.1013e-01,  3.1231e-01, -4.3369e-01, -2.8091e-01, -2.9654e-01,
        -5.8493e-01,  1.4303e-01, -2.4939e-01, -2.5161e-01,  6.0594e-03,
        -5.9773e-01, -4.3773e-01, -1.4739e-01,  4.8842e-01, -4.6687e-01,
        -7.2992e-01, -3.9896e-01,  2.1405e-01, -2.8487e-01,  1.6911e-02,
        -4.6969e-01, -3.9771e+00, -2.1120e-01, -3.4707e-01, -4.6502e-01,
        -4.9656e-02,  2.0078e-02,  6.4202e-01,  2.1692e-01, -7.6416e-02,
         6.8904e-01, -1.3983e-01, -2.0804e-01,  8.4225e-01,  1.2888e-01,
         1.2685e-01,  4.2034e-01, -3.5567e-01,  6.0724e-02,  1.0310e-01,
         7.3485e-01, -2.9214e-01,  5.2459e-01,  3.0

In [275]:
print(op[0][-1][:,0,:][2])

tensor([-4.0789e-01, -6.8875e-02,  3.6261e-01,  2.0112e-01, -1.5135e-01,
         2.8060e-02,  8.3402e-01,  1.0176e-02, -1.1804e-01,  5.2988e-02,
         1.7072e-01, -5.0285e-01,  1.6522e-02,  3.3955e-01,  6.9982e-02,
         1.1600e-01, -1.1538e-01,  6.1831e-01,  3.2471e-01, -1.5858e-02,
         1.4249e-01,  4.4013e-02,  5.2495e-01,  2.6497e-02, -2.9817e-01,
        -2.6367e-01,  3.4122e-02, -3.6504e-01,  2.9245e-01,  1.8539e-01,
        -4.0250e-01,  4.9998e-01,  2.1738e-01,  6.1250e-02,  4.8249e-01,
        -9.2055e-02, -5.2978e-02, -1.5313e-01,  7.0847e-01, -2.6938e-01,
         9.9794e-02, -2.6253e-01,  4.4361e-01,  2.2402e-02,  3.7130e-02,
        -4.5729e-01, -3.6381e+00,  1.3415e-02, -2.7874e-01, -3.1738e-01,
         7.8971e-03,  3.1639e-02,  6.3865e-01,  3.6483e-01, -5.1463e-01,
         4.7512e-01, -2.1788e-01,  1.0854e-01,  8.3495e-01, -1.5587e-01,
         1.7460e-01,  1.9191e-01, -1.7365e-01,  2.2614e-01, -3.8654e-01,
         5.2357e-01, -3.5761e-01,  5.2230e-02, -9.2

In [58]:
op[1]

tensor([[-0.7763, -0.4299, -0.9290,  ..., -0.6874, -0.6675,  0.7268],
        [-0.6634, -0.3750, -0.9179,  ..., -0.7056, -0.6269,  0.5975],
        [-0.6469, -0.3415, -0.9396,  ..., -0.6685, -0.5531,  0.7790]],
       grad_fn=<TanhBackward0>)

In [59]:
data_batch

['Do you consider yourself attractive?',
 'Do you consider yourself attractive?',
 'How should I kiss my girlfriend for the first time?']

In [60]:
op[1][2].shape

torch.Size([768])

In [61]:
op[1]

tensor([[-0.7763, -0.4299, -0.9290,  ..., -0.6874, -0.6675,  0.7268],
        [-0.6634, -0.3750, -0.9179,  ..., -0.7056, -0.6269,  0.5975],
        [-0.6469, -0.3415, -0.9396,  ..., -0.6685, -0.5531,  0.7790]],
       grad_fn=<TanhBackward0>)

In [62]:
for _, i in enumerate(tqdm(train_loader, desc="Training", leave=False)):
    print(i)
    print(test_trip_toks[0])
    break
#     op = bert(i)

tensor([[[  101,  2129,  2079,  1045,  3967,  1037,  2613, 23307,  1029,   102,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [  101,  2129,  2079,  1045,  3967,  1037,  2613, 23307,  1029,   102,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [  101,  2073,  2071,  1045,  2424,  1037, 17371,  3638,  4003,  2951,
           7233,  4007,  2489,  1997,  3465,  1029,   102,     0,     0,     0]]])
tensor([[ 101, 2054, 2024, 2070, 1997, 1996, 2190, 4219, 3784, 2000, 4553, 2446,
         1029,  102,    0,    0,    0,    0,    0,    0],
        [ 101, 2054, 2024, 2070, 1997, 1996, 2190, 4219, 3784, 2000, 4553, 2446,
         1029,  102,    0,    0,    0,    0,    0,    0],
        [ 101, 2129, 2079, 1045, 2131, 3825, 2005, 3015, 1029,  102,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]])


In [74]:
device = 'cpu'
epochs = 3

In [75]:
model.train()
for epoch in tqdm(range(epochs), desc="Epochs"):
    running_loss = []
    for step, txt_pcs in enumerate(tqdm(train_loader, desc="Training", leave=False)):
        txt_pcs = txt_pcs[0]
#         anchor_txt = txt_pcs[0][0,:]
#         positive_txt = txt_pcs[0][1,:]
#         negative_txt = txt_pcs[0][2,:]
#         anchor_txt = anchor_txt.to(device)
#         positive_txt = positive_txt.to(device)
#         negative_txt = negative_txt.to(device)
#         single_instance = [anchor_txt, positive_txt, negative_txt]
        
        optimizer.zero_grad()
        model_op = model(txt_pcs)
        anchor_out = model_op[0][-1][:,0,:][0]
        positive_out = model_op[0][-1][:,0,:][1] #model(positive_txt)
        negative_out = model_op[0][-1][:,0,:][2] #model(negative_txt)
        
        print(anchor_out.shape, negative_out.shape)
        
        loss = criterion(anchor_out, positive_out, negative_out)
        loss.backward()
        optimizer.step()
        
        running_loss.append(loss.cpu().detach().numpy())
    print("Epoch: {}/{} - Loss: {:.4f}".format(epoch+1, epochs, np.mean(running_loss)))

Training:   0%|          | 0/144 [00:00<?, ?it/s]

torch.Size([768]) torch.Size([768])



Training:   1%|          | 1/144 [00:01<02:34,  1.08s/it]

torch.Size([768]) torch.Size([768])



Training:   1%|▏         | 2/144 [00:01<02:17,  1.03it/s]

...

torch.Size([768]) torch.Size([768])



Training:  99%|█████████▉| 143/144 [00:57<00:00,  2.52it/s]

torch.Size([768]) torch.Size([768])



Epochs: 100%|██████████| 3/3 [02:54<00:00, 58.07s/it]      

Epoch: 3/3 - Loss: 6.5370


In [228]:
bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [117]:
bert = BertModel.from_pretrained('bert-base-uncased')

In [121]:
tok_text = torch.tensor(tokenizer.batch_encode_plus(data_batch, pad_to_max_length=True)['input_ids'])
tok_text

tensor([[  101,  2129,  2064,  1045,  3623,  1996,  3177,  1997,  2026,  4274,
          4434,  2096,  2478,  1037, 21210,  2078,  1029,   102],
        [  101,  2129,  2064,  1045,  3623,  1996,  3177,  1997,  2026,  4274,
          4434,   102,     0,     0,     0,     0,     0,     0]])

In [132]:
tok_text.shape
tok_text[0], tok_text[1]

(tensor([  101,  2129,  2064,  1045,  3623,  1996,  3177,  1997,  2026,  4274,
          4434,  2096,  2478,  1037, 21210,  2078,  1029,   102]),
 tensor([ 101, 2129, 2064, 1045, 3623, 1996, 3177, 1997, 2026, 4274, 4434,  102,
            0,    0,    0,    0,    0,    0]))

In [124]:
op = bert(tok_text)
type(op), len(op)

(tuple, 2)

In [128]:
type(op[0]), len(op[0]), len(op[1])

(list, 12, 2)

In [129]:
op

([tensor([[[ 0.1794,  0.2469, -0.0790,  ...,  0.0424,  0.0461,  0.0859],
           [-0.2570,  1.5673, -1.4244,  ..., -0.1042,  0.6053,  0.5434],
           [ 1.2409, -0.4284,  0.2041,  ..., -0.6638,  0.5053, -1.4993],
           ...,
           [-0.5067, -0.8269,  0.1051,  ..., -0.9189, -0.1751,  0.2254],
           [ 0.5423,  0.0294, -0.6308,  ...,  0.3701,  0.5269,  1.0042],
           [-0.0452,  0.0536,  0.1051,  ..., -0.2519,  0.0201, -0.1321]],
  
          [[ 0.2086, -0.0878, -0.1736,  ...,  0.0230,  0.0755,  0.1805],
           [ 0.0699,  1.6820, -1.2459,  ...,  0.5651, -0.2337,  0.1825],
           [ 1.2176, -0.2480,  0.3136,  ..., -1.2115,  0.3101, -1.1016],
           ...,
           [ 0.0293, -0.0316,  0.2374,  ...,  0.3078, -0.3459,  0.4025],
           [-0.0866,  0.0057,  0.2624,  ...,  0.0565,  0.2232,  0.2352],
           [ 0.0463, -0.0772,  0.2613,  ...,  0.1003, -0.2641,  0.2848]]],
         grad_fn=<AddBackward0>),
  tensor([[[ 1.4097e-01, -4.5585e-02, -1.9856e-01,  